# Matrix Factorization

The first model to compare with baseline is matrix factorization. Since we need to train implicit data, we consider Alternating Least Squares (ALS) as MF model which simple and truely competitive.

In [218]:
import os
import pickle

import numpy as np
import pandas as pd
from implicit.als import AlternatingLeastSquares
from scipy import sparse
from tqdm.auto import tqdm

from utils.metrics import *

## Load data

In [69]:
train = pd.read_parquet("./data/train.parquet")
valid = pd.read_parquet("./data/valid.parquet")
test = pd.read_parquet("./data/test.parquet")
most_common_beers = pd.read_parquet("./data/most_common_beers.parquet")

In [3]:
train_users = train["review_profilename"].unique()
valid_users = valid["review_profilename"].unique()
test_users = test["review_profilename"].unique()

In [178]:
baseline_topk = most_common_beers[:10]["beer_name"].tolist()

In [4]:
with open("./data/user_mapper.dict", "rb") as f:
    user_mapper = pickle.load(f)
with open("./data/item_mapper.dict", "rb") as f:
    item_mapper = pickle.load(f)

In [5]:
train_mat = np.zeros((len(user_mapper), len(item_mapper)))
valid_mat = np.zeros((len(user_mapper), len(item_mapper)))
test_mat = np.zeros((len(user_mapper), len(item_mapper)))

In [6]:
for _, user, item, rating in tqdm(train.itertuples(), total=len(train)):
    train_mat[user, item] = 1

for _, user, item, rating in tqdm(valid.itertuples(), total=len(valid)):
    valid_mat[user, item] = 1

for _, user, item, rating in tqdm(test.itertuples(), total=len(test)):
    test_mat[user, item] = 1

  0%|          | 0/1022381 [00:00<?, ?it/s]

  0%|          | 0/220993 [00:00<?, ?it/s]

  0%|          | 0/222265 [00:00<?, ?it/s]

## Modeling

In [ ]:
def predict(
    user_ids: np.ndarray,
    train_mat: sparse.csr_matrix,
    model: AlternatingLeastSquares,
    popular_items: list,
    top_k: int,
) -> np.ndarray:
    # Make recommendations based on the model
    rec = model.recommend(
        user_ids, train_mat[user_ids], N=top_k, filter_already_liked_items=True
    )

    # Substitutes for cold users with the most popular items
    rec_items = np.array(
        [
            popular_items if np.all(scores == 0) else items
            for items, scores in zip(rec[0], rec[1])
        ]
    )

    return rec_items

In [ ]:
train_csr = sparse.csr_matrix(train_mat)
valid_csr = sparse.csr_matrix(valid_mat)
test_csr = sparse.csr_matrix(test_mat)

In [209]:
# Hyper-parameters
factors = 128
regularization = 0.01

In [210]:
model = AlternatingLeastSquares(
    factors=factors,
    regularization=regularization,
    iterations=200,
    random_state=22,
)

In [211]:
model.fit(train_csr)

  0%|          | 0/200 [00:00<?, ?it/s]

In [234]:
test_users

array([31260,  4865, 23128, ..., 21243, 22791, 17589])

In [212]:
pred = predict(
    user_ids=test_users,
    train_mat=train_csr,
    model=model,
    popular_items=baseline_topk,
    top_k=10,
)

In [213]:
map_at_10 = map_at_k(actual=test_mat[test_users], pred=pred, top_k=10)

print(f"MAP@10 for baseline: {map_at_10:.6f}")

  0%|          | 0/9085 [00:00<?, ?it/s]

MAP@10 for baseline: 0.053626


## Prepare the hyperparameter tuning job

In [ ]:
REGION = "us-central1"

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

ARTIFACT_STORE = f"gs://{PROJECT_ID}-beer-artifact-store"

DATA_ROOT = f"{ARTIFACT_STORE}/data"
JOB_DIR_ROOT = f"{ARTIFACT_STORE}/jobs"
API_ENDPOINT = f"{REGION}-aiplatform.googleapis.com"

In [ ]:
os.environ["JOB_DIR_ROOT"] = JOB_DIR_ROOT
os.environ["PROJECT_ID"] = PROJECT_ID
os.environ["REGION"] = REGION

In [ ]:
!gsutil ls | grep ^{ARTIFACT_STORE}/$ || gsutil mb -l {REGION} {ARTIFACT_STORE}

Creating gs://qwiklabs-asl-04-5e165f533cac-beer-artifact-store/...


In [311]:
!gsutil -m cp ./data/* {DATA_ROOT}

Copying file://./data/item_mapper.dict [Content-Type=application/octet-stream]...
Copying file://./data/item_mapper.parquet [Content-Type=application/octet-stream]...
Copying file://./data/most_common_beers.parquet [Content-Type=application/octet-stream]...
Copying file://./data/train.parquet [Content-Type=application/octet-stream]...  
Copying file://./data/test.parquet [Content-Type=application/octet-stream]...   
Copying file://./data/user_mapper.dict [Content-Type=application/octet-stream]...
Copying file://./data/user_mapper.parquet [Content-Type=application/octet-stream]...
Copying file://./data/valid.parquet [Content-Type=application/octet-stream]...  
/ [8/8 files][ 16.7 MiB/ 16.7 MiB] 100% Done                                    
Operation completed over 8 objects/16.7 MiB.                                     


In [ ]:
from google.cloud import storage

In [ ]:
client = storage.Client()

In [ ]:
bucket = client.get_bucket(ARTIFACT_STORE[5:])

In [ ]:
blob = bucket.get_blob("data/user_mapper.dict")

In [306]:
pd.DataFrame.from_dict(user_mapper, orient="index").rename(columns={0: "idx"}).to_parquet("./data/user_mapper.parquet", index=True)

In [307]:
pd.DataFrame.from_dict(item_mapper, orient="index").rename(columns={0: "idx"}).to_parquet("./data/item_mapper.parquet", index=True)

In [310]:
pd.DataFrame.from_dict(user_mapper, orient="index").rename(columns={0: "idx"})["idx"].to_dict()

{'0110x011': 0,
 '01Ryan10': 1,
 '02maxima': 2,
 '03SVTCobra': 3,
 '04101Brewer': 4,
 '05Harley': 5,
 '0Naught0': 6,
 '0beerguy0': 7,
 '0runkp0s': 8,
 '0to15': 9,
 '0tt0': 10,
 '0xFF': 11,
 '1000Bottles': 12,
 '1001111.0': 13,
 '100floods': 14,
 '100proof': 15,
 '103stiga': 16,
 '104bob': 17,
 '1050Sudz': 18,
 '108Dragons': 19,
 '1099.0': 20,
 '10bear': 21,
 '10shb': 22,
 '1100.0': 23,
 '110toyourleft': 24,
 '1121987.0': 25,
 '11millsown113': 26,
 '11osixBrew': 27,
 '11soccer11': 28,
 '11thFloorBrewing': 29,
 '1229design': 30,
 '12647summerfield': 31,
 '12NattiBottles': 32,
 '12ouncecurls': 33,
 '12percent': 34,
 '12puebloyankee': 35,
 '12vUnion': 36,
 '12vman': 37,
 '130guy': 38,
 '13aphomet': 39,
 '13smurrf': 40,
 '159beerrunner': 41,
 '160Shillings': 42,
 '16ozSampler': 43,
 '17202826.0': 44,
 '1759Girl': 45,
 '1759dallas': 46,
 '17Guinness59': 47,
 '1844original': 48,
 '184601.0': 49,
 '187.0': 50,
 '18alpha': 51,
 '18todrink': 52,
 '18tony': 53,
 '196osh': 54,
 '1993Heel': 55,
 '1